<a href="https://colab.research.google.com/github/rboghe/cened/blob/master/cened_merged_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**TO-DO: filter before merging**

##Loading Files

In [40]:
import os
import urllib.request
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, make_scorer
from lightgbm import LGBMRegressor
import scipy.stats as stats
from sklearn.model_selection import RandomizedSearchCV

# Silence panda's setting with copy warning
pd.options.mode.chained_assignment = None

In [2]:
url = 'https://www.dati.lombardia.it/api/views/bbky-sde5/rows.csv?accessType=DOWNLOAD'

if os.path.isdir('/tmp'):
    if os.path.isfile('/tmp/cened20.csv'):
      pass
    else:
      urllib.request.urlretrieve(url, '/tmp/cened20.csv')
else:
  os.mkdir('/tmp')
  urllib.request.urlretrieve(url, '/tmp/cened20.csv')

In [3]:
url = 'https://www.dati.lombardia.it/api/views/rsg3-xhvk/rows.csv?accessType=DOWNLOAD'


if os.path.isfile('/tmp/cened12.csv'):
  pass
else:
  urllib.request.urlretrieve(url, '/tmp/cened12.csv')


In [4]:
url = 'https://raw.githubusercontent.com/rboghe/cened/master/degreedays.txt'

urllib.request.urlretrieve(url, '/tmp/ddh.csv')

ddh = pd.read_csv('/tmp/ddh.csv', usecols = ['comune','dd'])

In [5]:
cols20 = ['COMUNE','FOGLIO', 'PARTICELLA','SUPERFICIE_DISPERDENTE', 'EP_H_ND',
        'RIQUALIFICAZIONE_ENERGETICA','RISTRUTTURAZIONE_IMPORTANTE', 
        'INTERO_EDIFICIO','SUPERF_UTILE_RISCALDATA','EP_GL_NREN','EP_GL_REN']

cols12 = ['COMUNE','FOGLIO', 'PARTICELLA','SUPERFICIE_DISPERDENTE',
          'TRASMITTANZA_MEDIA_COPERTURA','TRASMITTANZA_MEDIA_INVOLUCRO',
          'TRASMITTANZA_MEDIA_BASAMENTO','TRASMITTANZA_MEDIA_SERRAMENTO',
          'SUPERFICIE_LORDA', 'VOLUME_LORDO', 'DESTINAZIONE_DI_USO',
          'ANNO_COSTRUZIONE','SUPERFICIE_VETRATA_OPACA', 'VOLUME_NETTO',
          'SUPERFICIE_NETTA', 'EPH']
  
cened20 = pd.read_csv('/tmp/cened20.csv', usecols = cols20)
cened12 = pd.read_csv('/tmp/cened12.csv', usecols = cols12)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Pre-filtering

In [6]:
cened20['EP_GL_NREN'].fillna(0, inplace=True)
cened20['EP_GL_REN'].fillna(0, inplace=True)

# Drop NaN 
cened20 = cened20.dropna()
cened12 = cened12.dropna() 

In [7]:
# Only entire buildings
cened20 = cened20[cened20['INTERO_EDIFICIO'] == True]

In [8]:
# Drop renovations
cened20 = cened20[cened20['RISTRUTTURAZIONE_IMPORTANTE'] == False]
cened20 = cened20[cened20['RIQUALIFICAZIONE_ENERGETICA'] == False]

## Merge

In [9]:
# Merge dfs
cenedtot = pd.merge(left=cened20, right=cened12, left_on=['COMUNE','FOGLIO', 'PARTICELLA'], 
                    right_on=['COMUNE','FOGLIO', 'PARTICELLA'],  how='inner') # , 'SUBALTERNO' | , 'SUBALTERNO'

In [10]:
# Compute difference between ext surfaces
cenedtot['surfdiff'] = abs((cenedtot['SUPERF_UTILE_RISCALDATA'] - cenedtot['SUPERFICIE_NETTA'])/cenedtot['SUPERF_UTILE_RISCALDATA'])

# Drop rows with high difference
cenedtot = cenedtot[cenedtot['surfdiff'] < 0.2]

# Compute difference between ext surfaces
#cenedtot['surfdiff'] = abs((cenedtot['SUPERFICIE_DISPERDENTE_y'] - cenedtot['SUPERFICIE_DISPERDENTE_x'])/cenedtot['SUPERFICIE_DISPERDENTE_y'])

# Drop rows with high difference
#cenedtot = cenedtot[cenedtot['surfdiff'] < 0.05]



In [11]:
# Compute difference between ext surfaces
cenedtot['ephdiff'] = abs((cenedtot['EPH'] - cenedtot['EP_H_ND'])/cenedtot['EPH'])

# Drop rows with high difference
cenedtot = cenedtot[cenedtot['ephdiff'] < 0.3]

In [12]:
cenedtot = cenedtot.dropna() 

##Feature Creation

In [13]:
# Add ddh
cenedtot['COMUNE'] = cenedtot['COMUNE'].str.lower()
ddh['comune'] = ddh['comune'].str.lower()
cenedtot['COMUNE'] = cenedtot['COMUNE'].str.replace(r"o`",  "o'")
cenedtot['COMUNE'] = cenedtot['COMUNE'].str.replace(r"baranzate",  "bollate")
cenedtot['COMUNE'] = cenedtot['COMUNE'].str.replace(r"bovisio masciago",  "bovisio-masciago")
cenedtot['COMUNE'] = cenedtot['COMUNE'].str.replace(r"cornate d`adda",  "cornate d'adda")
cenedtot['COMUNE'] = cenedtot['COMUNE'].str.replace(r"cortenuova",  "cortenova")
cenedtot['COMUNE'] = cenedtot['COMUNE'].str.replace(r"sant'omobono terme",  "sant'omobono imagna")
ddh['comune'] = ddh['comune'].str.replace(r"è",  "e'")
ddh['comune'] = ddh['comune'].str.replace(r"è",  "e'")
ddh['comune'] = ddh['comune'].str.replace(r"é",  "e'")
ddh['comune'] = ddh['comune'].str.replace(r"ò",  "o'")
ddh['comune'] = ddh['comune'].str.replace(r"ù",  "u'")

cenedtot = cenedtot.merge(ddh, left_on='COMUNE', right_on='comune', how='left')

In [14]:
cenedtot.ANNO_COSTRUZIONE = cenedtot.ANNO_COSTRUZIONE.astype(str)
cenedtot.ANNO_COSTRUZIONE = cenedtot.ANNO_COSTRUZIONE.map(lambda x:x[-4:])
cenedtot.ANNO_COSTRUZIONE = cenedtot.ANNO_COSTRUZIONE.astype(int)

cenedtot['PERIODO'] = pd.cut(cenedtot.ANNO_COSTRUZIONE, bins=[0,1930,1945,1960,1976,1992,2006,3000],
     labels=['Before 1930','1930-1945','1946-1960','1961-1976', '1977-1992', '1993-2006','After 2006']).cat.codes

In [15]:
# Opaque surface
cenedtot['SUP_OPACA'] = cenedtot['SUPERFICIE_DISPERDENTE_y']/(1+cenedtot['SUPERFICIE_VETRATA_OPACA'])

# Opaque surface
cenedtot['SUP_FINESTRATA'] = cenedtot['SUPERFICIE_DISPERDENTE_y'] - cenedtot['SUP_OPACA']

In [16]:
cenedtot['ALTEZZA_MEDIA'] = cenedtot['VOLUME_NETTO']/cenedtot['SUPERFICIE_NETTA']

In [17]:
cenedtot['TRASMITTANZA_MEDIA'] = (cenedtot['TRASMITTANZA_MEDIA_SERRAMENTO'] + cenedtot['TRASMITTANZA_MEDIA_COPERTURA'] +
         cenedtot['TRASMITTANZA_MEDIA_BASAMENTO'] + cenedtot['TRASMITTANZA_MEDIA_INVOLUCRO'])/4

In [18]:
cenedtot['EP_GL_TOT'] = cenedtot['EP_GL_REN'] + cenedtot['EP_GL_NREN']

##Filters

In [19]:
for col in ['TRASMITTANZA_MEDIA_COPERTURA','TRASMITTANZA_MEDIA_INVOLUCRO',
          'TRASMITTANZA_MEDIA_BASAMENTO','TRASMITTANZA_MEDIA_SERRAMENTO']:
          cenedtot = cenedtot[cenedtot[col].between(0,6, inclusive=False)]

In [20]:
# Drop buildings with wrong An
cenedtot = cenedtot[cenedtot['SUPERFICIE_NETTA'] > 50]

# Drop buildings with wrong Vn
cenedtot = cenedtot[cenedtot['VOLUME_NETTO'] > 150]

In [21]:
cenedtot = cenedtot[cenedtot['DESTINAZIONE_DI_USO'] == 'E.1(1)']

In [22]:
cenedtot = cenedtot[cenedtot['TRASMITTANZA_MEDIA'] > 0.15]
cenedtot = cenedtot[cenedtot['TRASMITTANZA_MEDIA'] < 4]

In [23]:
cenedtot = cenedtot[cenedtot['ALTEZZA_MEDIA'] > 2.4]
cenedtot = cenedtot[cenedtot['ALTEZZA_MEDIA'] < 6]

In [24]:
# Drop buildings with wrong EPH
# cenedtot = cenedtot[cenedtot['EP_H_ND'] > 120]
# cenedtot = cenedtot[cenedtot['EP_H_ND'] < 1000]

In [25]:
len(cenedtot)

1121

# Shuffle

In [53]:
cened = cenedtot.sample(frac=1, random_state = 42).reset_index(drop=True)

# Reserve a test set

In [54]:
msk = np.random.rand(len(cened)) < 0.8
train = cened[msk]
test = cened[~msk]

In [55]:
print(len(train))

898


In [56]:
print(len(test))

223


# Define MAPE

In [57]:
def neg_mape(y_true, y_pred):
  y_true = np.array(y_true)
  y_pred = np.array(y_pred)
  return -np.abs((y_true - y_pred)/y_true).mean()

In [58]:
def modified_neg_mape(y_true, y_pred):
  y_true = np.array(y_true)
  y_pred = np.array(y_pred)
  errors = np.abs((y_true - y_pred)/y_true)
  index = np.argwhere(errors > 1)
  mod_errors = np.delete(errors, index)
  return -mod_errors.mean()

In [59]:
neg_mape_scorer = make_scorer(neg_mape, greater_is_better=True)

In [60]:
modified_neg_mape_scorer = make_scorer(modified_neg_mape, greater_is_better=True)

# Random search

In [61]:
mlcol = ['SUPERFICIE_DISPERDENTE_y', 'SUPERFICIE_NETTA', 'VOLUME_NETTO',
          'TRASMITTANZA_MEDIA_COPERTURA','TRASMITTANZA_MEDIA_INVOLUCRO',
          'TRASMITTANZA_MEDIA_BASAMENTO','TRASMITTANZA_MEDIA_SERRAMENTO',
          'SUPERFICIE_LORDA', 'VOLUME_LORDO','dd','PERIODO','SUP_OPACA',
        'SUP_FINESTRATA']

In [62]:
cat_feats = ['PERIODO']

In [63]:
param_dist = {'feature_fraction': np.linspace(0.4, 1, num=7),
              'num_leaf' : list(range(20,40)),
             'max_depth' : [-1],
             'max_bin' : [100, 200, 300, 500, 750, 1000, 2000],
             'bagging_fraction' : np.linspace(0.4, 1, num=14),
             'bagging_freq' : list(range(1,10)),
             'lambda_l1' : stats.uniform(0, 0.6),
             'lambda_l2' : stats.uniform(0, 0.6)}

lgbm = LGBMRegressor(n_estimators = 100, silent = True, verbose = 0, is_training_metric = True, n_jobs = 1, 
                     eval_metric  = 'mape')

n_iter_search = 100

random_search = RandomizedSearchCV(lgbm, param_distributions=param_dist, n_iter=n_iter_search, 
                                   scoring=modified_neg_mape_scorer, cv = 5, n_jobs = 6, verbose = 2)

random_search.fit(cened[mlcol], cened['EP_H_ND'], categorical_feature = cat_feats)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:    3.7s
[Parallel(n_jobs=6)]: Done 150 tasks      | elapsed:   17.4s
[Parallel(n_jobs=6)]: Done 353 tasks      | elapsed:   41.4s
[Parallel(n_jobs=6)]: Done 500 out of 500 | elapsed:   58.8s finished
/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['PERIODO']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=LGBMRegressor(boosting_type='gbdt',
                                           class_weight=None,
                                           colsample_bytree=1.0,
                                           eval_metric='mape',
                                           importance_type='split',
                                           is_training_metric=True,
                                           learning_rate=0.1, max_depth=-1,
                                           min_child_samples=20,
                                           min_child_weight=0.001,
                                           min_split_gain=0.0, n_estimators=100,
                                           n_jobs=1, num_leaves=31,
                                           objective=None, random_sta...
                                        'lambda_l1': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f603f0da470>,
            

# Print results

In [64]:
print(random_search.best_params_)

{'bagging_fraction': 1.0, 'bagging_freq': 9, 'feature_fraction': 0.8, 'lambda_l1': 0.3302609136328864, 'lambda_l2': 0.22570682954991958, 'max_bin': 1000, 'max_depth': -1, 'num_leaf': 37}


In [65]:
print(random_search.best_score_)

-0.18207162534881421


In [66]:
# Create a results df
cv_df = pd.DataFrame(random_search.cv_results_)
cv_df = cv_df.sort_values(by = ['rank_test_score']).reset_index()

# Filter columns
res_cols = [col for col in cv_df if col.startswith('split')]

# Select best iter
best_res = cv_df.loc[0]

# Print results for each fold
print(best_res[res_cols])

split0_test_score   -0.193338
split1_test_score   -0.181844
split2_test_score   -0.178311
split3_test_score   -0.171965
split4_test_score     -0.1849
Name: 0, dtype: object


# Train the final model

In [67]:
light = LGBMRegressor(n_estimators = 1000, silent = False, verbose = 2, is_training_metric = True, n_jobs = 6)
light.set_params(**random_search.best_params_)

LGBMRegressor(bagging_fraction=1.0, bagging_freq=9, boosting_type='gbdt',
              class_weight=None, colsample_bytree=1.0, feature_fraction=0.8,
              importance_type='split', is_training_metric=True,
              lambda_l1=0.3302609136328864, lambda_l2=0.22570682954991958,
              learning_rate=0.1, max_bin=1000, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=1000, n_jobs=6, num_leaf=37, num_leaves=31,
              objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,
              silent=False, subsample=1.0, subsample_for_bin=200000,
              subsample_freq=0, verbose=2)

In [68]:
light.fit(train[mlcol], train['EP_H_ND'])

LGBMRegressor(bagging_fraction=1.0, bagging_freq=9, boosting_type='gbdt',
              class_weight=None, colsample_bytree=1.0, feature_fraction=0.8,
              importance_type='split', is_training_metric=True,
              lambda_l1=0.3302609136328864, lambda_l2=0.22570682954991958,
              learning_rate=0.1, max_bin=1000, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=1000, n_jobs=6, num_leaf=37, num_leaves=31,
              objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,
              silent=False, subsample=1.0, subsample_for_bin=200000,
              subsample_freq=0, verbose=2)

# Test the model

In [69]:
y_pred = light.predict(test[mlcol])

In [70]:
results_df = pd.DataFrame({'predicted' : y_pred, 'true' : test['EP_H_ND']})
results_df['error'] = np.abs((results_df['true'] - results_df['predicted'])/results_df['true'])*100

In [71]:
# MAPE
results_df['error'].mean()

18.85386508049255

In [72]:
# MAPE without >100%
results_df[results_df.error <= 100].error.mean()

18.15812668920175

In [73]:
results_df

,predicted,true,error
11,104.087323,113.24,8.082548
21,218.983908,194.62,12.518707
26,180.055105,98.03,83.673473
28,196.301958,217.36,9.688094
43,154.343370,189.37,18.496399
...,...,...,...
1109,160.789049,99.04,62.347586
1111,217.176818,308.42,29.584068
1113,107.054636,125.47,14.677105
1115,138.600574,138.57,0.022064
